In [2]:
cd /content/drive/MyDrive/DeepLearningTasks/17FlowersClassification

/content/drive/MyDrive/DeepLearningTasks/17FlowersClassification


In [ ]:
!pip install wandb

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
wandb.init(project='17FlowersClassification')
config=wandb.config
config.learning_rate=0.0001

accuracy,0.85407
best_epoch,5
best_val_loss,1.10269
epoch,9
loss,0.47593
val_accuracy,0.65686
val_loss,1.2743


accuracy,▁▄▅▆▇▇▇███
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▂▁▁▁
val_accuracy,▁▃▅▇▇█▇██▇
val_loss,█▅▃▂▁▁▂▁▁▂


In [23]:
dataset_path = '/content/drive/MyDrive/Datasets/Flowers/Flowers'
width = height = 224
batch_size = 16

data_generator = ImageDataGenerator(
    rescale = 1/.255,
    horizontal_flip =True,
    rotation_range=20,
    zoom_range=0.1,
    validation_split=0.2
)

train_data = data_generator.flow_from_directory(
    os.path.join(dataset_path, "Train"),
    target_size=(width, height),
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)


val_data = data_generator.flow_from_directory(
    os.path.join(dataset_path, "Train"),
    target_size=(width, height),
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

print(np.bincount(train_data.labels))
print(np.bincount(val_data.labels))

Found 884 images belonging to 17 classes.
Found 204 images belonging to 17 classes.
[52 52 52 52 52 52 52 52 52 52 52 52 52 52 52 52 52]
[12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12]


In [24]:
# train_images = next(train_data)[0]
# plt.figure(figsize=(8,8))

# for i in range(16):
#   plt.subplot(4,4,i+1)
#   plt.xticks([])
#   plt.yticks([])
#   plt.grid(False)
#   plt.imshow(train_images[i],cmap=plt.cm.binary)

In [25]:
# base_model = tf.keras.applications.VGG19(
#     input_shape=(width, height, 3),
#     include_top=False,
#     weights='imagenet'
# )

# base_model.summary()

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(width, height, 3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=17,
    classifier_activation="softmax",
    # **kwargs
)
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [26]:
for layer in base_model.layers:
  layer.trainable=False

In [27]:
model = tf.keras.Sequential([
            base_model,
            Flatten(),
            # Dense(512, activation='relu'),
            # Dropout(0.2),
            # Dense(128, activation='relu'),
            Dense(17, activation='softmax')
])

In [28]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
                             metrics=['accuracy'])

In [29]:
model.fit(train_data,
          steps_per_epoch=train_data.samples/batch_size, #884/32
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size, #204/32
          epochs=15,
          callbacks=[WandbCallback()])


Epoch 1/15
55/55 [==============================] - 25s 407ms/step - loss: 2.7431 - accuracy: 0.2670 - val_loss: 1.9482 - val_accuracy: 0.4461


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/15
55/55 [==============================] - 21s 382ms/step - loss: 1.4674 - accuracy: 0.5452 - val_loss: 1.5067 - val_accuracy: 0.5490
Epoch 3/15
55/55 [==============================] - 21s 383ms/step - loss: 1.2714 - accuracy: 0.6109 - val_loss: 1.5740 - val_accuracy: 0.6029
Epoch 4/15
55/55 [==============================] - 22s 403ms/step - loss: 0.9908 - accuracy: 0.6957 - val_loss: 1.2598 - val_accuracy: 0.6324
Epoch 5/15
55/55 [==============================] - 21s 383ms/step - loss: 0.7999 - accuracy: 0.7568 - val_loss: 1.1805 - val_accuracy: 0.6618
Epoch 6/15
55/55 [==============================] - 21s 387ms/step - loss: 0.7298 - accuracy: 0.7794 - val_loss: 1.1331 - val_accuracy: 0.6814
Epoch 7/15
55/55 [==============================] - 21s 389ms/step - loss: 0.6081 - accuracy: 0.8066 - val_loss: 1.2163 - val_accuracy: 0.6667
Epoch 8/15
55/55 [==============================] - 21s 388ms/step - loss: 0.5416 - accuracy: 0.8428 - val_loss: 1.2518 - val_accuracy: 0.6961

In [30]:
test_data_path = '/content/drive/MyDrive/Datasets/Flowers/Flowers/Test'

data_generator = ImageDataGenerator(rescale = 1/.255)
test_data = data_generator.flow_from_directory(
    test_data_path,
    target_size=(width, height),
    batch_size = batch_size,
    class_mode='categorical')

model.evaluate(test_data)

Found 272 images belonging to 17 classes.
17/17 [==============================] - 3s 153ms/step - loss: 2.5941 - accuracy: 0.3750


[2.5940513610839844, 0.375]